In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [11]:
import os
from PIL import Image
import numpy as np

# Define the directories
mask_dir = 'data\with_mask'
without_mask_dir = 'data\without_mask'

# Initialize lists to store the images and labels
# Initialize lists to store the images and labels
images = []
labels = []

# Process images with masks
for filename in os.listdir(mask_dir):
    img = Image.open(os.path.join(mask_dir, filename))
    if img.mode == 'RGB' and img.size == (224, 224):
        img = img.resize((224, 224))
        images.append(np.array(img))
        labels.append(1)  # Assign label '1' for images with masks

# Process images without masks
for filename in os.listdir(without_mask_dir):
    img = Image.open(os.path.join(without_mask_dir, filename))
    if img.mode == 'RGB' and img.size == (224, 224):
        img = img.resize((224, 224))
        images.append(np.array(img))
        labels.append(0)  # Assign label '0' for images without masks
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


In [12]:
# Check the shape of each image
for i, img in enumerate(images):
    if img.shape != (224, 224, 3):
        print(f"Image index {i} shape: {img.shape}")


In [13]:
images[0].shape

(224, 224, 3)

In [14]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Now, X_train and y_train are your training data and labels
# X_test and y_test are your testing data and labels


In [16]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# Download the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Make the model non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Add a new top layer
x = base_model.output
x = Flatten()(x)  # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)  # New FC layer
predictions = Dense(1, activation='sigmoid')(x)  # New softmax layer

# Create a new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


In [20]:
from tensorflow import keras


In [26]:
#from tensorflow.keras.models import load_model

# Load the model from the file
model = keras.models.load_model('mask_model.h5')


In [27]:
import cv2
import numpy as np

# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# To capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    _, img = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face = np.expand_dims(face, axis=0)
        pred = model.predict(face)[0][0]
        if pred == 0:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(img, 'No Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        else:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, 'Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    # Display
    cv2.imshow('img', img)

    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break

# Release the VideoCapture object
cap.release()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 992ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
h